# Imports

In [2]:
import numpy as np
import pandas as pd
import spacy
nlp = spacy.load('en_core_web_sm')

import time
from scipy.special import gammaln
from collections import Counter
from collections import defaultdict
from matplotlib import pyplot as plt

# Import stopwords
import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')

# Task 1

## Setup

In [3]:
def docs_to_int(corpus:str, encoding:str, nr_tokens:int, stop:list) -> (list(), Counter()):
    
    token_count = 0
    docs = list()
    docs_count = 0
    
    # Find unique words (removal of stopwords and punctuations and other stuff.)
    freqs = Counter()   
    
    with open(corpus, encoding = encoding) as f:
        
        while token_count < nr_tokens:
            
            doc = f.readline()
            docs_count += 1
            temp_doc = list()
            
            # Lemmatizion of tokens
            result = nlp(doc.lower(), disable=['tagger', 'parser', 'ner'])
            
            for token in result:
                if len(token.lemma_) > 2:
                    if token.lemma_ not in stop:
                        token_count += 1
                        freqs[token.lemma_] += 1
                        temp_doc.append(token.lemma_)
                        
            docs.append(temp_doc)

    print('Nr of tokens:' + str(token_count))
    print('Nr of docs:' + str(docs_count))
    
    # Create bag of words
    bow = []
    vocab = freqs.most_common()

    for i in range(len(vocab)):
        bow.append(vocab[i][0])

    # Get pairs of elements    
    tmp = zip(bow, range(0,len(vocab)-1))
    # Make pairs into a dictionary
    vocab = dict(tmp)
        
    # Match token to int
    docs_int = list()
    
    for doc in docs:
        docs_int.append(list([vocab.get(x) for x in doc]))
        
    return docs_int, freqs, vocab

In [4]:
#stop = list(string.punctuation)
stop = list()

for x in stopwords.words('english'):
    stop.append(x)

'''
Add a couple of more stopwords which were observed manually
'''
for x in ['-PRON-','\n','...','..',"'d'","n't"]:
    stop.append(x)
    

## Code

In [5]:
# Define variables

corpus = 'books.txt'
encoding = 'ISO-8859-1'
M = 10**5    # Corpus size

In [24]:
docs_int, freqs, vocab = docs_to_int(corpus=corpus, encoding=encoding, nr_tokens=M, stop = stop)
vocab_size = len(freqs)

Nr of tokens:100030
Nr of docs:1237


In [25]:
nan_loc = []

for d, line in enumerate(docs_int):

    for j, w in enumerate(line):
        if w == None:
            nan_loc.append((d, j))
        
nan_loc

[(1236, 99)]

In [26]:
temp_doc = []

with open(corpus, encoding = encoding) as f:
    for line_idx, line in enumerate(f):
        if line_idx == d:
            print(line)
            
            result = nlp(line.lower(), disable=['tagger', 'parser', 'ner'])
            
            for token in result:
                if len(token.lemma_) > 2:
                    if token.lemma_ not in stop:
                        
                        temp_doc.append(token.lemma_)
                             
            for idx, token in enumerate(temp_doc):
                print(idx, token)

it 's amazing how despite all the tragedies and wars , big business elitists are able to cash in on the damage while religious fundamentalists never get caught , much less held accountable . the idiots who show their hate of this book are from terrorist nations that have a knack of socializing poverty and terrorism while at the same time privatizing wealth . despite all the big talk about winning the so-called war on terrorism , the ugly truth is wars have not taught us anything . if it were n't for big business funding hitler , hitler would have had a harder time killing the jews . sadly though , even after world war ii ended , the big business elites that funded and continue to fund dictatorships like hitler , stalin , and the modern ones are not only not held accountable but often end up walking away as " heroes " . if we 're really going to win the war on terrorism and / or poverty , we 're going to have to stop supporting big business elite and stop allowing our uber-corrupt polit

In [27]:
# Quick fix - sort this out later

docs_int[nan_loc[0][0]][nan_loc[0][1]] = 0

In [28]:
freqs.most_common()[0:10]

[('book', 2795),
 ('much', 1163),
 ('read', 1097),
 ('one', 834),
 ('good', 736),
 ('like', 540),
 ('would', 524),
 ('write', 522),
 ('make', 488),
 ('story', 476)]

In [29]:
docs_int[10]

[17, 3, 0, 2121, 255, 6919, 1912, 21, 5, 116, 394, 6920, 3824, 2740]

In [30]:
vocab["astral"]

4869

## LDA STUFF

### Functions

In [32]:
def initialize_lda(docs_int, freqs, n_topics):
    
    n_docs = len(docs_int)
    vocab_size = len(freqs)

    # Word topic in doc, corresponds to \Theta
    ndz = np.zeros((n_docs,n_topics))

    # Word topic count, corresponds to \phi
    nzw = np.zeros((n_topics, vocab_size))

    # Counters for documents and topics
    nd = np.zeros(n_docs)
    nz = np.zeros(n_topics)

    # Create dictionary of topics
    topics = {}

    # iterate over documents 
    for d, line in enumerate(docs_int):
        for i, w in enumerate(line):
                  
            # w = docs_int[d][i] # Numerical rep of word w in doc d

            # Initialise with a random topic
            z = np.random.randint(n_topics)
            topics[(d,i)] = z

            # Increase counters
            ndz[d, z] += 1
            nzw[z, w] += 1

            nd[d] += 1
            nz[z] += 1

    return topics, ndz, nzw, nd, nz

In [34]:
def cond_topic_prob(ndz, nzw, nz, nd, w, d, alpha, beta, n_topics, voc_size):
    """
    Conditional probability of topics. 
    """

    left = (nzw[:,w] + beta) / (nz + beta * voc_size)
    right = (ndz[d,:] + alpha) / (nd[d] + alpha * n_topics)

    p_z = left * right
    p_z /= np.sum(p_z)
    
    return p_z

In [35]:
def log_multinomial_beta(alpha, K=None):

    if K is None:
        # alpha is assumed to be a vector
        return np.sum(gammaln(alpha)) - gammaln(np.sum(alpha))
    else:
        # alpha is assumed to be a scalar
        return K * gammaln(alpha) - gammaln(K*alpha)


In [43]:
def loglikelihood(n_topics, voc_size, alpha, beta, nzw, ndz):
    likelihood = 0
    
    n_docs = ndz.shape[0]
    
    for z in range(n_topics):
        likelihood += log_multinomial_beta(nzw[z,:] + beta)
        likelihood -= log_multinomial_beta(beta, voc_size)
        
    for d in range(n_docs):
        likelihood += log_multinomial_beta(ndz[d,:] + alpha)
        likelihood -= log_multinomial_beta(alpha, n_topics)
        
    return likelihood

In [54]:
def LDA_Gibbs_Sampler(docs_int, voc_size, n_topics, max_iterations, alpha, beta, freqs):

    start_time = time.time()
    topics, ndz, nzw, nd, nz = initialize_lda(docs_int, freqs, n_topics)

    for i in range(max_iterations):
        for d, line in enumerate(docs_int):
             for j, w in enumerate(line):
                
                z = topics[(d, j)]
                ndz[d, z] -= 1
                nzw[z, w] -= 1
                nd[d] -= 1
                nz[z] -= 1
            
                p_z = cond_topic_prob(ndz, nzw, nz, nd, w, d, alpha, beta, n_topics, vocab_size)
                z = np.random.multinomial(1, p_z).argmax() # maybe don't use argmax, use it as a probability
                
                ndz[d,z] += 1
                nzw[z,w] += 1
                nd[d] += 1
                nz[z] += 1
                topics[(d, j)] = z

        if i % 10 == 0:
            print("Iteration", i)
            print("Likelihood", loglikelihood(n_topics, voc_size, alpha, beta, nzw, ndz))

    elapsed_time = time.time() - start_time
    print("Elapsed time: ", elapsed_time)
    
    return nzw

In [46]:
def show_words_by_topic(word_topic_prob, vocabulary, typical_len):
    
    n_topics = word_topic_prob.shape[0]
    typical_words = []

    for i in range(n_topics):
        arr = word_topic_prob[i,:]
        typical_ints = arr.argsort()[-typical_len-2:-2][::-1]   # there's some funny business with the last word in vocab
        #print(typical_ints)

        for search_int in typical_ints:
            if search_int in [0, -1]:
                typical_words.append("")
            else:
                for k, v in vocabulary.items(): 
                    if v == search_int:
                        typical_words.append(k)
                        break

    # Print the most common words in each topic
    typical_words = np.reshape(typical_words, [n_topics, -1])
    print(typical_words)

### Code

In [33]:
alpha = 0.1  # Hyperparameter
beta = 0.1   # Hyperparameter
K = 10       # Nr of topics
it = 100      # Nr of iterations

topics, ndz, nzw, nd, nz = initialize_lda(docs_int, freqs, K)

In [57]:
K = 10
alpha = 0.1
beta = alpha

word_topic_prob_01_10, log_lik_01_10 = LDA_Gibbs_Sampler(docs_int, vocab_size, K, it, alpha, beta, freqs)
show_words_by_topic(word_topic_prob_01_10, vocab, 10)

Iteration 0
Likelihood -1032872.9336507256
Iteration 10
Likelihood -922503.9558998622
Iteration 20
Likelihood -901413.2203634626
Iteration 30
Likelihood -890514.91921006
Iteration 40
Likelihood -883245.3888710137
Elapsed time:  273.2125232219696
[['much' 'good' 'like' 'get' 'write' 'one' 'would' 'make' 'time' 'great']
 ['much' 'like' 'one' 'good' 'character' 'would' 'get' 'story' 'little'
  'say']
 ['author' 'church' 'name' 'historical' 'spiritual' 'word' 'view' 'human'
  'doe' 'science']
 ['man' 'old' 'world' 'time' 'life' 'like' 'year' 'family' 'young' 'play']
 ['novel' 'read' 'much' 'reader' 'one' 'life' 'character' 'write' 'work'
  'live']
 ['take' 'government' 'prospero' 'much' 'race' 'force' 'james' 'live'
  'give' 'success']
 ['read' 'much' 'use' 'find' 'one' 'great' 'work' 'think' 'write' 'many']
 ['food' 'news' 'smith' 'show' 'new' 'get' 'eat' 'buy' 'find' 'include']
 ['american' 'history' 'man' 'write' 'account' 'america' 'military' 'one'
  'power' 'human']
 ['much' 'one' 'st

In [58]:
K = 50
alpha = 0.1
beta = alpha

word_topic_prob_01_50, log_lik_01_50 = LDA_Gibbs_Sampler(docs_int, vocab_size, K, it, alpha, beta, freqs)
show_words_by_topic(word_topic_prob_01_50, vocab, 10)

Iteration 0
Likelihood -1181398.7504743172
Iteration 10
Likelihood -993165.9162597334
Iteration 20
Likelihood -949611.1432695037
Iteration 30
Likelihood -928661.2743395902
Iteration 40
Likelihood -918175.8819436037
Iteration 50
Likelihood -911789.5153736168
Iteration 60
Likelihood -906753.2238608436
Iteration 70
Likelihood -902399.8762865227
Iteration 80
Likelihood -898857.2788436102
Iteration 90
Likelihood -896018.964296845
Elapsed time:  580.6504461765289
[['collection' 'baseball' 'selection' 'hal' 'bach' 'musical' 'rock'
  'player' 'christians' 'hitter']
 ['vedic' 'religious' 'iskcon' 'coelho' 'veronika' 'god' 'beauty'
  'plague' 'technology' 'patient']
 ['law' 'britain' 'find' 'webb' 'nine' 'well' 'pendel' 'laymon' 'john'
  'thirty']
 ['news' 'america' 'country' 'government' 'political' 'medium' 'power'
  'military' 'state' 'show']
 ['subject' 'eastern' 'snake' 'battle' 'yates' 'politic' 'map' 'benefit'
  'cultural' 'cop']
 ['biography' 'guralnick' 'cooke' 'cancer' 'version' 'push'

In [55]:
K = 10
alpha = 0.01
beta = alpha

word_topic_prob_001_10 = LDA_Gibbs_Sampler(docs_int, vocab_size, K, it, alpha, beta, freqs)
show_words_by_topic(word_topic_prob_001_10, vocab, 10)

Iteration 0
Likelihood -1068254.9497986077
Iteration 10
Likelihood -935168.9056323167
Iteration 20
Likelihood -920033.2654612741
Iteration 30
Likelihood -912652.8357973326
Iteration 40
Likelihood -908021.0522482244
Elapsed time:  281.6496891975403
[['like' 'would' 'character' 'one' 'think' 'much' 'get' 'story' 'good'
  'make']
 ['read' 'one' 'good' 'get' 'little' 'great' 'love' 'story' 'child'
  'work']
 ['one' 'man' '' 'may' 'american' 'problem' 'make' 'well' 'would' 'use']
 ['read' 'people' 'time' 'would' 'find' 'good' 'make' 'think' 'idea'
  'many']
 ['history' 'food' 'issue' 'write' 'interest' 'much' 'come' 'want' 'also'
  'look']
 ['read' 'write' 'good' 'one' 'great' 'think' 'find' 'make' 'character'
  'well']
 ['love' 'become' 'human' 'work' 'family' 'even' 'write' 'life' 'mean'
  'year']
 ['people' 'novel' 'good' 'say' 'world' 'give' 'create' 'person' 'part'
  'much']
 ['one' 'story' 'much' 'would' 'way' 'two' 'life' 'news' 'people' 'call']
 ['good' 'much' 'one' 'like' 'find' 'k

In [56]:
K = 50
alpha = 0.01
beta = alpha

word_topic_prob_001_50 = LDA_Gibbs_Sampler(docs_int, vocab_size, K, it, alpha, beta, freqs)
show_words_by_topic(word_topic_prob_001_50, vocab, 10)

Iteration 0
Likelihood -1189673.8378284348
Iteration 10
Likelihood -952077.5114507346
Iteration 20
Likelihood -929264.4139531485
Iteration 30
Likelihood -919213.3685814547
Iteration 40
Likelihood -912071.2629240489
Elapsed time:  289.19020915031433
[['get' 'one' 'life' 'help' 'new' 'feel' 'read' 'end' 'thing' 'give']
 ['much' 'one' 'problem' 'use' 'help' 'work' 'good' 'need' 'look'
  'reader']
 ['use' 'little' 'give' 'different' 'event' 'would' 'history' 'probably'
  'may' 'find']
 ['use' 'guide' 'world' 'gibbon' 'life' 'novel' 'find' 'guralnick' 'two'
  'biography']
 ['art' 'great' 'way' 'take' 'actually' 'life' 'artist' 'year' 'love'
  'learn']
 ['read' 'find' 'would' 'really' 'say' 'author' 'story' 'point' 'make'
  'know']
 ['think' 'people' 'god' 'game' 'belief' 'well' 'end' 'still' 'one'
  'follow']
 ['know' 'doe' 'world' 'one' 'get' 'come' 'tell' 'live' 'say' 'year']
 ['country' 'world' 'year' 'power' 'fight' 'show' 'name' 'political'
  'military' 'many']
 ['write' 'character' 'o